In [2]:
%cd ..

/home/phongtnh/Person-Search/person_rlf


In [6]:
file_path = "/home/phongtnh/Person-Search/vn3k_person_search/VN3K/imgs/TEST_DATA/Person1.2/output_2.jpg"
vi_captions = "người đàn_ông tóc cắt ngắn , mặc áo_phông màu xám nhạt có chữ màu đỏ trên ngực áo , quần màu xám nhạt , và đi_đôi giày thể_thao màu đen với dây giày màu đỏ ."
en_captions = "The man had short hair, wore a light gray T-shirt with red letters on the chest, light gray pants, and black sneakers with red shoelaces."

In [11]:
from model.siglip.modeling_siglip import SiglipVisionModel

model = SiglipVisionModel.from_pretrained('/home/phongtnh/Person-Search/person_rlf/siglip_checkpoints')

In [12]:
model

SiglipVisionModel(
  (vision_model): SiglipVisionTransformer(
    (embeddings): SiglipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
      (position_embedding): Embedding(256, 768)
    )
    (encoder): SiglipEncoder(
      (layers): ModuleList(
        (0-11): 12 x SiglipEncoderLayer(
          (self_attn): SiglipAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): SiglipMLP(
            (activation_fn): PytorchGELUTanh()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=T

In [13]:
from PIL import Image
from model.siglip.image_processing_siglip import SiglipImageProcessor

processor = SiglipImageProcessor.from_pretrained('/home/phongtnh/Person-Search/person_rlf/siglip_checkpoints')
inputs = processor(Image.open(file_path), return_tensors="pt")

In [14]:
inputs

{'pixel_values': tensor([[[[-0.3020, -0.2706, -0.2941,  ..., -0.7176, -0.7333, -0.7412],
          [-0.3176, -0.3333, -0.3725,  ..., -0.7255, -0.7333, -0.7412],
          [-0.1451, -0.1843, -0.2235,  ..., -0.7333, -0.7333, -0.7412],
          ...,
          [-0.1451, -0.1608, -0.2314,  ...,  0.2078,  0.3961, -0.2863],
          [-0.1529, -0.1451, -0.2314,  ...,  0.1059,  0.3020, -0.3098],
          [-0.1608, -0.1373, -0.2235,  ...,  0.0431,  0.1922, -0.3176]],

         [[-0.6314, -0.5922, -0.6157,  ..., -0.8196, -0.8196, -0.8275],
          [-0.6392, -0.6549, -0.6863,  ..., -0.8196, -0.8196, -0.8275],
          [-0.4510, -0.4980, -0.5294,  ..., -0.8196, -0.8196, -0.8275],
          ...,
          [-0.0510, -0.0667, -0.1451,  ...,  0.2549,  0.4431, -0.2392],
          [-0.0588, -0.0510, -0.1451,  ...,  0.1529,  0.3490, -0.2627],
          [-0.0667, -0.0431, -0.1373,  ...,  0.0902,  0.2392, -0.2706]],

         [[-0.6471, -0.6078, -0.6392,  ..., -0.8667, -0.8510, -0.8588],
          [-0

In [15]:
inputs.pixel_values.shape

torch.Size([1, 3, 384, 128])

In [16]:
import torch
with torch.no_grad():
    outputs = model(**inputs, interpolate_pos_encoding=True)
outputs

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-2.1495, -0.4388, -1.6633,  ...,  2.0281, -0.8127,  0.6085],
         [-5.0171, -2.7226, -2.5749,  ..., -0.2247, -1.7627,  1.9544],
         [-0.1994, -2.6476,  0.1012,  ..., -2.8361,  1.2211,  3.4346],
         ...,
         [-0.3645,  0.0398, -1.1744,  ...,  0.0483, -0.2507, -0.0863],
         [ 2.3085, -0.7375,  1.2040,  ...,  0.2259, -1.8292, -2.2350],
         [ 1.1640, -0.1936,  1.8980,  ..., -0.6886, -0.3480, -2.0360]]]), pooler_output=tensor([[ 4.3088e-01,  9.3276e-02,  5.2031e-01, -2.9667e-01, -2.3218e-01,
          5.0049e-01,  1.3070e-01,  1.3383e-01, -1.5079e-01, -2.1903e-03,
          7.3598e-03, -1.9927e-01, -3.1792e-01,  7.3743e-01, -4.9895e-01,
          3.9500e-02, -1.4580e-01, -1.8776e-02, -1.2477e-01,  3.4925e-01,
         -1.8927e-01, -2.0986e-01,  7.6412e-02,  3.0814e-02, -2.0037e-01,
         -3.8367e-01, -5.2844e-01,  1.4947e-01,  8.4984e-02,  4.2733e-02,
         -2.7477e-01, -3.4590e-01, -2.9267e-01,  5.465

In [18]:
outputs.pooler_output.shape

torch.Size([1, 768])

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/home/phongtnh/Person-Search/person_rlf/siglip_checkpoints", model_max_length=187)
# input_text = tokenizer(vi_captions, return_tensors="pt", truncation=True, padding="max_length", return_attention_mask=True, max_length=100)

In [13]:
tokenizer.added_tokens_decoder

{1: AddedToken("</s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
 2: AddedToken("<unk>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True)}

In [11]:
tokenizer.added_tokens_decoder.update()

In [8]:
tokenizer.pad_token_id

1

In [9]:
input_text

{'input_ids': tensor([[ 1672,   355,  2353,  1735, 31029,   297,   317,  2289,   270,   594,
          6701,   326,  1335,   297,   259,   471,  2567, 78066,  1452,   273,
           259,   329,  4428,   677,  1262,   270,   885,   562,  2342,  1452,
           273,   355,  2071,   534,  1079,   594,  1849,   259,   471,   268,
           819,  1313,  1452,   273,   259,   329,  4428,   677,  1262,   270,
           259,   793,  3767, 11095,  3571,   718, 48066,   394,   924, 71949,
          1452,   273,   355,   278,   300,   908,   331,  2625,   718, 48066,
          1452,   273,   355,  2071,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1

In [203]:
tokenizer.decode(input_text.input_ids[0])

'người đànông tóc cắt ngắn mặc áophông màu xám nhạt có chữ màu đỏ trên ngực áo quần màu xám nhạt và điđôi giày thểthao màu đen với dây giày màu đỏ</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>'

In [208]:
input_text

{'input_ids': tensor([[ 1672,   355,  2353,  1735, 31029,   297,   317,  2289,   270,   594,
          6701,   326,  1335,   297,   259,   471,  2567, 78066,  1452,   273,
           259,   329,  4428,   677,  1262,   270,   885,   562,  2342,  1452,
           273,   355,  2071,   534,  1079,   594,  1849,   259,   471,   268,
           819,  1313,  1452,   273,   259,   329,  4428,   677,  1262,   270,
           259,   793,  3767, 11095,  3571,   718, 48066,   394,   924, 71949,
          1452,   273,   355,   278,   300,   908,   331,  2625,   718, 48066,
          1452,   273,   355,  2071,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1

In [5]:
tokenizer("<mask>", return_tensors="pt")

{'input_ids': tensor([[250000,      1]])}

In [10]:
# Replace the token with id 25000 with <mask> and flag it as special token
# {
#     "<mask>": 250000
# }
tokenizer.get_vocab().update({"<mask>": 250000})

In [2]:
tokenizer.convert_ids_to_tokens(250000)

'<mask>'

In [3]:
tokenizer.convert_tokens_to_ids("<mask>")

250000

In [4]:
# add mask token to the tokenizer
tokenizer.add_special_tokens({'mask_token': '<mask>'})

0

In [14]:
tokenizer.mask_token

'<mask>'

In [16]:
# change the mask token id to 250000
tokenizer.mask_token_id

250000

In [17]:
tokenizer.convert_tokens_to_ids("<mask>")

250000

In [125]:
from transformers import DataCollatorForLanguageModeling


data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15)

In [126]:
data_collator.torch_mask_tokens(input_text.input_ids)

(tensor([[  1672, 250000,   2353, 250000,  31029,    297,    317, 250000,    270,
             594,   6701,    326,   1335, 250000,    259,    471,   2567,  78066,
            1452, 250000,    259,    329,   4428,    677,   1262,    270, 250000,
          250000, 250000, 250000,    273,    355,   2071,    534,   1079,    594,
            1849, 250000, 250000,    268,    819,   1313,   1452,    273,    259,
             329,   4428,    677,   1262,    270,    259,    793,   3767, 250000,
            3571, 250000, 250000,    394, 250000,  71949,   1452,    273,    355,
             278,    300,    908,    331,   2625,    718,  48066,   1452,    273,
             355,   2071,      1]]),
 tensor([[ -100,  -100,  -100,  1735,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  1335,   297,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,   562,  -100,  1452,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,   259,  

In [3]:
from model.siglip.modeling_siglip import SiglipTextModel

text_model = SiglipTextModel.from_pretrained('/home/phongtnh/Person-Search/person_rlf/siglip_checkpoints')

/home/phongtnh/miniconda3/envs/person_search/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
text_model.config.hidden_size

768

In [128]:
import torch
with torch.no_grad():
    text_outputs = text_model(**input_text)
text_outputs.last_hidden_state.shape

IndexError: index out of range in self

In [ ]:
tokenizer.mask_token

In [215]:
from typing import Any, Optional
mlm_probability = 0.15
import torch

def torch_mask_tokens(inputs: Any, special_tokens_mask: Optional[Any] = None):
    """
    Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original.
    """
    labels = {
        "input_ids": inputs["input_ids"].clone(),
        "attention_mask": inputs["attention_mask"].clone(),
    }

    # We sample a few tokens in each sequence for MLM training (with probability `mlm_probability`)
    probability_matrix = torch.full(labels['input_ids'].shape, mlm_probability)
    if special_tokens_mask is None:
        special_tokens_mask = [
            tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels['input_ids'].tolist()
        ]
        special_tokens_mask = torch.tensor(special_tokens_mask, dtype=torch.bool)
    else:
        special_tokens_mask = special_tokens_mask.bool()

    probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels['input_ids'][~masked_indices] = 1  # We only compute loss on masked tokens

    # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = torch.bernoulli(torch.full(labels['input_ids'].shape, 0.8)).bool() & masked_indices
    inputs['input_ids'][indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    # 10% of the time, we replace masked input tokens with random word
    indices_random = torch.bernoulli(torch.full(labels['input_ids'].shape, 0.5)).bool() & masked_indices & ~indices_replaced
    random_words = torch.randint(len(tokenizer), labels['input_ids'].shape, dtype=torch.long)
    inputs['input_ids'][indices_random] = random_words[indices_random]

    # The rest of the time (10% of the time) we keep the masked input tokens unchanged
    return inputs, labels

inputs, labels = torch_mask_tokens(input_text)

In [216]:
inputs

{'input_ids': tensor([[  1672,    355,   2353,   1735,  31029,    297,    317,   2289,    270,
            594,   6701,    326,   1335,    297,    259,    471,   2567,  78066,
           1452,    273,    259,    329,   4428,    677,   1262,    270,    885,
            562,   2342,   1452, 250000,    355,   2071,    534, 250000,    594,
           1849,    259,    471,    268,    819, 250000,   1452,    273,    259,
            329,   4428, 250000,   1262, 250000,    259,    793, 250000,  11095,
         250000,    718,  48066,    394,    924,  71949, 250000,    273,    355,
            278,    300,    908, 128128,   2625,    718,  48066,   1452,    273,
            355,   2071,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [217]:
labels['input_ids'].reshape(-1)

tensor([   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,  273,    1,    1,    1, 1079,    1,
           1,    1,    1,    1,    1, 1313,    1,    1,    1,    1,    1,  677,
           1,  270,    1,  793, 3767,    1, 3571,    1,    1,    1,    1,    1,
        1452,    1,    1,    1,    1,    1,  331,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1])

In [194]:
tokenizer.decode(inputs['input_ids'][0])

'người đ<mask> ông tóc cắt ng<mask><mask> ặc áophông màu <mask> ám nhạt có chữ màu<mask> ỏ trên ngực áo quần<mask> u<mask> xám<mask> ạ<mask> và đi<mask> ôi gimuč thểthao<mask> u<mask><mask> với dây giày màu đỏ</s>'

In [ ]:
import json 

with open("/home/phongtnh/Person-Search/vn3k_person_search/VN3K/data_captions.json") as f:
    data = json.load(f)
    
data[0]

{'': 0,
 'id': 1,
 'file_path': 'TEST_DATA/Person1.2/output_2.jpg',
 'split': 'train',
 'captions': ['người đàn_ông tóc cắt ngắn , mặc áo_phông màu xám nhạt có chữ màu đỏ trên ngực áo , quần màu xám nhạt , và đi_đôi giày thể_thao màu đen với dây giày màu đỏ .',
  'người đàn_ông dáng đi vai hơi gù , mặc bộ quần_áo màu xám nhạt , sải bước chân rộng với đôi giày màu đen thắt dây màu đỏ .'],
 'processed_tokens': [['người',
   'đàn_ông',
   'tóc',
   'cắt',
   'ngắn',
   ',',
   'mặc',
   'áo_phông',
   'màu',
   'xám',
   'nhạt',
   'có',
   'chữ',
   'màu',
   'đỏ',
   'trên',
   'ngực',
   'áo',
   ',',
   'quần',
   'màu',
   'xám',
   'nhạt',
   ',',
   'và',
   'đi_đôi',
   'giày',
   'thể_thao',
   'màu',
   'đen',
   'với',
   'dây',
   'giày',
   'màu',
   'đỏ',
   '.'],
  ['người',
   'đàn_ông',
   'dáng',
   'đi',
   'vai',
   'hơi',
   'gù',
   ',',
   'mặc',
   'bộ',
   'quần_áo',
   'màu',
   'xám',
   'nhạt',
   ',',
   'sải',
   'bước',
   'chân',
   'rộng',
   'với',
   'đô

In [ ]:
# find the maximum length of the captions
max = 0
for d in data:
    captions = d['captions']
    en_captions = d['en_captions']
    # for cap in captions:
    #     l = tokenizer(cap, return_tensors="pt").input_ids.shape[1]
    #     if l > max:
    #         max = l
    for cap in en_captions:
        l = tokenizer(cap, return_tensors="pt").input_ids.shape[1]
        if l > max:
            max = l

max

118

In [ ]:
import sentencepiece as spm
s = spm.SentencePieceProcessor(model_file='/home/phongtnh/Person-Search/person_rlf/siglip_checkpoints/spiece.model')

In [ ]:
len(s)

250100